In [1]:
import sys
sys.path.append('../')
import utils
import models
import numpy as np 
import matplotlib.pyplot as plt 
import cv2
import tensorflow as tf 
 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

Using TensorFlow backend.
1 Physical GPUs, 1 Logical GPUs


In [2]:
 x_data, y_data = utils.load_dataset_sice()
    
data_keys = list(x_data.keys())

num_data = len(x_data)
last_index_train = int(num_data * 0.7)
last_index_val = int(num_data * 0.9)

x_train = models.split_dictionary(x_data, data_keys[:last_index_train])
y_train = models.split_dictionary(y_data, data_keys[:last_index_train])

x_val = models.split_dictionary(x_data,  data_keys[last_index_train: last_index_val])
y_val = models.split_dictionary(y_data,  data_keys[last_index_train: last_index_val])

x_test = models.split_dictionary(x_data,  data_keys[last_index_val: -1])
y_test = models.split_dictionary(y_data,  data_keys[last_index_val: -1])



In [3]:
index = 10
keys = list(x_test.keys())
x_test_img = cv2.imread(x_test[keys[index]])     
y_test_img = cv2.imread(y_test[keys[index]]) 

# resize_
height, width = x_test_img.shape[:2]

x_test_img = cv2.resize(x_test_img, dsize=(width//4, height//4), interpolation=cv2.INTER_CUBIC) 
y_test_img = cv2.resize(y_test_img, dsize=(width//4, height//4), interpolation=cv2.INTER_CUBIC) 
# epand dim
x_test_img = np.expand_dims(x_test_img, axis=0)
cv2.imwrite('./test_result/img.ref.bmp',y_test_img)

True

In [4]:
ITERATIONS = 10
BATCH_SIZE = 4

train_generator = models.data_generator_from_path(x_train, y_train, batch_size=BATCH_SIZE)
# val_generator = models.data_generator_from_path(x_val, y_val, batch_size=BATCH_SIZE)

model = models.build_model(None, None, 3)
model.compile(optimizer='adam', loss='mse', metrics=['acc'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, None, None, 64)    256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, None, None, 64)    256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 128)   7385

In [5]:
start = 0
for step in range(10000):
    x_batch, y_batch = next(train_generator)
    loss = model.train_on_batch(x_batch, y_batch)

    if step % 100 == 0:
        model.save_weights('./weights/ae_model.{0}.h5'.format(step))
        print('epochs: {0}, loss: {1:.2f}, acc: {2:.2f}'.format(step, loss[0], loss[1]))
        # inference (prediction)
        im = model.predict(x_test_img/255.0)
        im = im.squeeze()
        cv2.imwrite('./test_result/img.{0}.bmp'.format(step), np.uint8(im * 255))
        
       

epochs: 0, loss: 0.07, acc: 0.36
epochs: 100, loss: 0.04, acc: 0.28
epochs: 200, loss: 0.03, acc: 0.61
epochs: 300, loss: 0.04, acc: 0.21
epochs: 400, loss: 0.04, acc: 0.25
epochs: 500, loss: 0.07, acc: 0.38
epochs: 600, loss: 0.04, acc: 0.24
epochs: 700, loss: 0.02, acc: 0.54
epochs: 800, loss: 0.06, acc: 0.39
epochs: 900, loss: 0.03, acc: 0.34
epochs: 1000, loss: 0.06, acc: 0.51
epochs: 1100, loss: 0.04, acc: 0.26
epochs: 1200, loss: 0.04, acc: 0.38
epochs: 1300, loss: 0.03, acc: 0.34
epochs: 1400, loss: 0.03, acc: 0.41
epochs: 1500, loss: 0.03, acc: 0.45
epochs: 1600, loss: 0.03, acc: 0.32
epochs: 1700, loss: 0.02, acc: 0.39
epochs: 1800, loss: 0.04, acc: 0.62
epochs: 1900, loss: 0.02, acc: 0.24
epochs: 2000, loss: 0.03, acc: 0.42
epochs: 2100, loss: 0.05, acc: 0.22
epochs: 2200, loss: 0.03, acc: 0.56
epochs: 2300, loss: 0.02, acc: 0.13
epochs: 2400, loss: 0.03, acc: 0.41
epochs: 2500, loss: 0.04, acc: 0.22
epochs: 2600, loss: 0.02, acc: 0.48
epochs: 2700, loss: 0.04, acc: 0.44
epoc